# <p style="color:hotpink">**AI for Attendance: Understanding No-Shows in Maternal Virtual Visits**


## <p style="color:hotpink">**Week 4 Update: Cleaning the Dataset**

This section focuses on cleaning and preparing the maternal telehealth no-show dataset for exploratory data analysis (EDA) and machine learning. Key steps include stadnardizing column formats, handling missing values, and engineering helpful features like message length. The result is a cleaned dataset ready for further analysis and modeling.

---

## <p style="color:hotpink">**Load and Inspect the Dataset**

The dataset was loaded using pandas, and the intiial shape was printed to verify row and column counts. The first few records were previewed using .head() to understand the structure and content.

In [ ]:
# Load the datatset
import pandas as pd

df = pd.read_csv("maternal_telehealth_full_dataset_final.csv")
print(df.shape)
df.head()

(10150, 18)


,patient_id,patient_name,patient_age,zip_code,insurance,referral_source,visit_type,booking_days_prior,confirmed_2_days_prior,ignored_all_outreach,past_visits,past_no_shows,credit_card_on_file,baby_age_months,status,patient_message,message_intent,stress_level
0,bdd640fb-0667-4ad1-9c80-317fa3b1799d,Courtney Doyle,32,95544,Major Insurance,Provider Referred,New Patient Postpartum,1,False,True,3,0,False,10.0,Show,NaN,NaN,NaN
1,16419f82-8b9d-4434-a465-e150bd9c66b3,Amanda Davis,21,55477,Major Insurance,Provider Referred,New Patient Prenatal,11,False,True,5,0,False,NaN,Show,NaN,NaN,NaN
2,8fadc1a6-06cb-4fb3-9a1d-e644815ef6d1,Marie Gardner,30,32906,Major Insurance,Provider Referred,New Patient Prenatal,17,False,True,2,0,True,NaN,Show,NaN,NaN,NaN
3,cf36d58b-4737-4190-96da-1dac72ff5d2a,Olivia Moore,23,85866,Major Insurance,Provider Referred,New Patient Prenatal,3,True,False,0,0,True,NaN,Show,NaN,NaN,NaN
4,371ecd7b-27cd-4130-8722-9389571aa876,Gabrielle Davis,32,70199,Major Insurance,DME Referred,Regular Care,0,False,True,2,0,False,8.0,Cancelled,NaN,NaN,NaN


## <p style="color:hotpink">**Standardize Column Names**

All column names were cleaned by stripping whitespace, converting to lowercase, and replacing spaces with underscores. This improves consistency and avoids common errors during feature referencing.

In [2]:
# Standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# <p style="color:hotpink">**Handle Missing Values**

1. Categorical columns "Patient_message", "message_intent", and "stress_level" were filled with the placeholder "None" to preserve row counts while marking missingness.
2. The numerical column "baby_age_months" was filled with 0 to represent unborn babies in prenatal scenarios.
3. No rows were dropped, maintaining the full dataset for analysis.

In [3]:
# Check missing values
missing = df.isnull().sum()
print("Missing values:\n", missing[missing > 0])

Missing values:
 baby_age_months    4551
patient_message    9650
message_intent     9650
stress_level       9650
dtype: int64


In [4]:
# Fill missing categorical with a label
cat_cols = ["patient_message", "message_intent", "stress_level"]
df[cat_cols] = df[cat_cols].fillna("None")

# Fill missing numerical fields
df["baby_age_months"] = df["baby_age_months"].fillna(0)

# Can drop rows with missing target or essential features also - not necessary/wanted in this case

# Print data head to check this new update
df.head()

,patient_id,patient_name,patient_age,zip_code,insurance,referral_source,visit_type,booking_days_prior,confirmed_2_days_prior,ignored_all_outreach,past_visits,past_no_shows,credit_card_on_file,baby_age_months,status,patient_message,message_intent,stress_level
0,bdd640fb-0667-4ad1-9c80-317fa3b1799d,Courtney Doyle,32,95544,Major Insurance,Provider Referred,New Patient Postpartum,1,False,True,3,0,False,10.0,Show,None,None,None
1,16419f82-8b9d-4434-a465-e150bd9c66b3,Amanda Davis,21,55477,Major Insurance,Provider Referred,New Patient Prenatal,11,False,True,5,0,False,0.0,Show,None,None,None
2,8fadc1a6-06cb-4fb3-9a1d-e644815ef6d1,Marie Gardner,30,32906,Major Insurance,Provider Referred,New Patient Prenatal,17,False,True,2,0,True,0.0,Show,None,None,None
3,cf36d58b-4737-4190-96da-1dac72ff5d2a,Olivia Moore,23,85866,Major Insurance,Provider Referred,New Patient Prenatal,3,True,False,0,0,True,0.0,Show,None,None,None
4,371ecd7b-27cd-4130-8722-9389571aa876,Gabrielle Davis,32,70199,Major Insurance,DME Referred,Regular Care,0,False,True,2,0,False,8.0,Cancelled,None,None,None


# <p style="color:hotpink">**Check Boolean Columns**

The columns "confirmed_2_days_prior", "ignored_all_outreach", and "credit_card_on_file" were confirmed to be of proper Boolean (True/False) type. No conversions were necessary.

In [5]:
# Checking True/False for true Booleans
print(df[["confirmed_2_days_prior", "ignored_all_outreach", "credit_card_on_file"]].dtypes)

confirmed_2_days_prior    bool
ignored_all_outreach      bool
credit_card_on_file       bool
dtype: object


# <p style="color:hotpink">**Remove Duplicates**

All exact duplicate rows were dropped using drop_duplicates() to ensure data integrity.

In [6]:
# Remove obvious duplicates
df = df.drop_duplicates()

# <p style="color:hotpink">**Fix Invalid Values**

Records with negative "baby_age_months" were remvoed, as they respresent biologically invalid entries.

In [7]:
# Check and fix invalid values

# Remove unrealistic baby ages
df = df[df["baby_age_months"] >= 0]

# <p style="color:hotpink">**Feature Engineering: Message Length**

A new column, "message_length", was created by counting the number of words in "patient_message". The placeholder "None" was excluded to avoid artificially inflating word counts.

In [8]:
# Add message length as a column for text EDA and treat "None" (the placeholder string) as missing and count length correctly
df["message_length"] = df["patient_message"].replace("None", "").fillna("").apply(lambda x: len(x.split()))

# Print df to check changes
df.head()

,patient_id,patient_name,patient_age,zip_code,insurance,referral_source,visit_type,booking_days_prior,confirmed_2_days_prior,ignored_all_outreach,past_visits,past_no_shows,credit_card_on_file,baby_age_months,status,patient_message,message_intent,stress_level,message_length
0,bdd640fb-0667-4ad1-9c80-317fa3b1799d,Courtney Doyle,32,95544,Major Insurance,Provider Referred,New Patient Postpartum,1,False,True,3,0,False,10.0,Show,None,None,None,0
1,16419f82-8b9d-4434-a465-e150bd9c66b3,Amanda Davis,21,55477,Major Insurance,Provider Referred,New Patient Prenatal,11,False,True,5,0,False,0.0,Show,None,None,None,0
2,8fadc1a6-06cb-4fb3-9a1d-e644815ef6d1,Marie Gardner,30,32906,Major Insurance,Provider Referred,New Patient Prenatal,17,False,True,2,0,True,0.0,Show,None,None,None,0
3,cf36d58b-4737-4190-96da-1dac72ff5d2a,Olivia Moore,23,85866,Major Insurance,Provider Referred,New Patient Prenatal,3,True,False,0,0,True,0.0,Show,None,None,None,0
4,371ecd7b-27cd-4130-8722-9389571aa876,Gabrielle Davis,32,70199,Major Insurance,DME Referred,Regular Care,0,False,True,2,0,False,8.0,Cancelled,None,None,None,0


# <p style="color:hotpink">**Final Checks + Saving New CSV**

The cleaned dataset's shape was printed, showing how many records remained.
The final cleaned dataset was saved as "maternal_telehealth_data_clean.csv" for use in future analysis and modeling steps.

In [9]:
# See how many records survived cleaning
print(f"Cleaned dataset has {df.shape[0]} rows and {df.shape[1]} columns.")

# Check for duplicates, if you plan to drop them
print("Duplicate rows:", df.duplicated().sum())


Cleaned dataset has 10150 rows and 19 columns.
Duplicate rows: 0


In [ ]:
# Save the dataset
df.to_csv("maternal_telehealth_data_clean.csv", index=False)


# <p style="color:hotpink">**Next Steps**

Next week will involve

1. Exploratory data analysis (EDA), including visualizations of patient, appointment, and message trends
2. Distribution plots, bar charts, timelines, and word cloud analysis

<p style="color:hotpink">Stay tuned for the next update!